# Complete LLM Hankbook

Install Dependencies to run code blocks

In [ ]:
%pip install tensorflow keras keras_nlp matplotlib numpy pandas scikit-learn

## Sections 

### 1 Tokenization, vocab, sequence formatting
- 1.1 Byte level words vs Subword tokenization
  - 1.1.1 Byte level tokenization
  - 1.1.2 Subword tokenization (BPE, WordPiece, SentencePiece)
- 1.2 Token Ids and Vocabulary Size
- 1.3 Padding and Masking
- 1.4 Special Tokens
- 1.5 Sequence Packing and Contiguous Streams
- 1.6 Sliding Window Chunking
- 1.7 Complete Example: Combining All Tokenization Steps

### 2 Embedding and Unembedding
- 2.1 Word embedding lookup tables
- 2.2 Unembedding and tied embeddings
  - 2.2.1 Unembedding
  - 2.2.2 Tied Embeddings
- 2.3 Why positional representations are required
- 2.4 Positional encoding types
  - 2.4.1 Sinusoidal positional encoding
  - 2.4.2 Learned positional embeddings
  - 2.4.3 Rotary Position Embeddings (RoPE)
  - 2.4.4 ALiBi
- 2.5 How positional encoding interacts with attention
- 2.6 Embedding scaling by sqrt(d_model)
- 2.7 Complete Example: Combining All Embedding Steps

### 3 Attention
- 3.1 Query, Key, Value fundamentals
- 3.2 Dot product attention
- 3.3 Why divide by √dk
- 3.4 Causal masking and autoregressive behavior
- 3.5 Softmax details and numerical stability
- 3.6 Multi-head attention
- 3.7 Attention complexity
- 3.8 Memory layout and tensor shapes
- 3.9 Flash Attention (conceptual)
- 3.10 Self attention vs cross attention
- 3.11 Key-value caching for generation
- 3.12 TensorFlow Examples
  - 3.12.1 Single-head attention
  - 3.12.2 Multi-head attention
  - 3.12.3 Causal mask
  - 3.12.4 Multi-head attention demo
  - 3.12.5 What transformers do: context integration

## 1 Tokenization, vocab, sequence formatting

### 1.1 Byte level words vs Subword tokenization 

**Why**: Transformers cannot process raw text, text must be converted into numbers. The way we break text into tokens affects efficency, generalization and memory usage

#### 1.1.1 Byte level tokenization

- works at the byte level (0-255)
- Real world usage: GPT-2 uses byte pair encoding (BPE) at byte level
- Pros:
  - Handels any charecter, any language, emojis, symbol
  - no OOV (out of vocab) tokens
- Cons:
  - Toekn sequences can be longer -> means more compute 
- Example: "hello 👋" –> [104, 101, 108, 108, 111, 32, 240, 159, 145, 139] (in token ids where range is 0 -> vocab_size)

#### 1.1.2 Subword tokenization (BPE, WordPeice, SentencePiece)

-  Breaks Text into frequent subwords insted of characters or words. 
-  Example: 
   -  "unhappiness" -> ["un", "happi", "ness"] -> [217, 9812, 403] # in token ids (range is 0 -> vocab size)
- Pros: 
  - Shorter sequences than byte
  - Can handle rare words via subword decomposition (breaking unknown words into known smaller parts)
- Cons:
  - some complexity in building vocab and handling edge cases
  
**NOTE:** LLM's often use subword BPE (BPE applied at the subword level) it iteratively merges the most frequent character or subword pairs to build a vocabulary, balancing between character-level and word-level tokenization

### 1.2 Token Ids and Vocabulary Size
- After Tokenization, each token is mapped to a integer ID using a vocabulary
- Vocabulary size (V) is very important
  - Larger V -> model must have a bigger embedding matrix (page 50 in written notes) -> more parameters (hence a larger model)
  - Smaller V -> more subword splitting (words broken into more pieces) -> longer sequences -> slower training (but smaller model size)
- Typical LLM vocab sizes: 30K-100K for english models 
- Example: In TensorFlow, keras_nlp.tokenizers handles both mapping tokens → IDs and IDs → tokens.

``` py
from keras_nlp.tokenizers import BytePairTokenizer

tokenizer = BytePairTokenizer(vocabulary=["hello", "world", "un", "happi", "ness", "<PAD>", "<BOS>", "<EOS>"])
tokens = tokenizer.tokenize(["hello world", "unhappiness"])
token_ids = tokenizer(tokens)
print(token_ids)

```

**How Keras NLP Tokenizers Handle Token ↔ ID Mapping** Under the hood, Keras NLP tokenizers maintain two key data structures (`token_to_id` and `id_to_token`) for bidirectional mapping. When you call `tokenizer.tokenize(text)`, it returns tokens as strings; `tokenizer(text)` returns token IDs; and `tokenizer.detokenize(ids)` converts IDs back to text. The vocabulary is built during training or loaded from a pre-trained model, with special tokens (PAD, UNK, BOS, EOS) typically assigned fixed IDs at the beginning.


### 1.3 Padding and Masking 
1. Padding: Short sequences are extended with PAD tokens to match the longest sequence in a batch, enabling efficient parallel processing (e.g., `[5, 10, 15]` → `[5, 10, 15, <PAD>, <PAD>]`)

2. Attention Masking: Tells the transformer which positions to ignore during attention.
- **No Mask (Bidirectional)**: All tokens attend to all tokens; used in BERT for full context understanding
- **Causal Mask (Autoregressive)**: Each token only attends to previous tokens; used in GPT to prevent future information leakage during training
- **Padding Mask**: Masks PAD tokens so they don't affect attention scores; combined with other masks in most models

``` py
import tensorflow as tf

# Example: batch of token IDs (here each array of token ids in a batch is a sqeuence i.e one example, by spliting in batches we can proccess in parallel)
batch = tf.ragged.constant([
    [1, 2, 3],
    [4, 5]
])
padded = batch.to_tensor(default_value=0) # Output: [[1, 2, 3], [4, 5, 0]]  <- 0 is the PAD token ID (these are the new tokens)
mask = tf.cast(padded != 0, tf.int32) # Output: [[1, 1, 1], [1, 1, 0]]  <- tells attention to ignore the last position in sequence 2 (this is the attention scores not token values)
```

### 1.4 Special Tokens (see ML notes page 121)
- `<BOS>`: Beginning of sequence (marks where a sequence starts)
- `<EOS>`: End of sequence (marks where a sequence ends)
- `<PAD>`: Padding (fills shorter sequences to match batch length)
- `<UNK>`: Unknown/ out of vocab token (represents words not in vocabulary)
- etc

**usage in training**
``` text
Input:  <BOS> hello world <EOS> <PAD> <PAD>    # BOS is fed as a conditioning token ((a special input token that provides initial context/prompt for the model; the model conditions its next-token predictions on it but is not trained to predict it)) EOS is included so the model learns to predict sequence end PADs fill to uniform length

Target: hello world <EOS> <PAD> <PAD> <PAD>   # Target = input shifted left (model predicts the next token at each step, including EOS); PADs fill to uniform length

Mask:   1 1 1 1 0 0 0                          # Mask=1 for positions to compute loss (we compute loss for real tokens and EOS), 0 for PADs
```

### 1.5 Sequence Packing and Contiguous Streams 
- Why: LLM training is compute-heavy, to use memory efficiently, multiple short examples can be concatenated into a single long sequence and then chunked
- Benefits: 
  - Reduces wasted padding
  - keepinh sequences dense for attention
  
**Example (pseudo)**
```text
Examples: ["hello", "world"], ["goodbye", "moon"]
Packed sequence: "hello world goodbye moon"
```
- Then split into fixed length chunks (ex: 8 tokens per chunk) for processing

### 1.6 Sliding Window Chunking 
- When text is too long to fit in memory, we create overlapping windown to preserve context
- why: prevents cutting off dependencies between sequences.
- Example Sequence length = 6, chunk size = 4, stride = 2
``` text
Sequence: [A B C D E F] (len = 6)

Chunk 1: Start at position 0 → [A B C D] (len = 4 beacuse chunk size = 4)
Chunk 2: Start at position 0 (position) + 2(stride) = 2 → [C D E F] (move the window by 2, keeps last 2 tokens of last chink in new chunk)

Chunks:  [A B C D], [C D E F]

Result Sequence: [A B C D], [C D E F]
                      ↑overlap↑
```
- Edge case: If the final window doesn't have enough tokens (e.g., only 3 tokens left for chunk_size=4), you either pad it with `<PAD>` tokens or discard it depending on your training strategy.
- overlapping ensures context continuity for training 
- common in LLM pretraining 
  

### 1.7 Complete Example: Combining All Tokenization Steps

This example demonstrates the entire pipeline from raw text to training-ready sequences, incorporating all concepts from 1.1-1.6.

In [11]:
import tensorflow as tf
import keras_nlp

# Load pretrained tokenizer (handles vocab, special tokens, BPE subword)
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")

# Create dataset pipeline: tokenization → chunking → padding
dataset = (
    tf.data.Dataset.from_tensor_slices(["Hello world", "Goodbye moon"])
    .map(tokenizer)  # Tokenize: text → IDs
    .unbatch()  # Flatten to token stream (packing)
    .batch(8, drop_remainder=False)  # Chunk into sequences of 8 tokens
    .map(lambda x: (x[:-1], x[1:]))  # Create (input, target) pairs
    .padded_batch(2, padded_shapes=([None], [None]))  # Pad and batch
)

# Usage:
inputs, targets = next(iter(dataset))
print(tokenizer.detokenize(inputs[0]))
print(tokenizer.detokenize(targets[0]))

Hello worldGoodbye
 worldGoodbye moon


**OUTPUT:**
```
Hello worldGoodbye
worldGoodbye moon
```

**EXPLANATION:**

The dataset variable is a `tf.data.Dataset` pipeline that transforms raw text into training-ready (input, target) pairs. It's a lazy iterator (doesn't process until called).

**Pipeline steps:**
1. **from_tensor_slices**: Creates dataset from list of strings
2. **map(tokenizer)**: Converts each text → token IDs (e.g., "Hello" → [15496, 995])
3. **unbatch()**: Flattens all sequences into one continuous token stream (sequence packing)
4. **batch(8)**: Groups tokens into chunks of 8 (creates fixed-length sequences)
5. **map(lambda)**: Splits each chunk into (input, target) where target = input shifted left
6. **padded_batch(2)**: Groups 2 sequences into a batch, pads shorter ones to match length

**How next-token prediction works:**

The model predicts the NEXT token at EACH position, not just the last one:
- Position 0: Given "Hello" → predict "world"
- Position 1: Given "Hello world" → predict "Goodbye"  
- Position 2: Given "Hello worldGoodbye" → predict "moon"

So the target sequence shows what should be predicted at each step. The entire target = input shifted left by 1 token (each target is the next token).

**Chunking Strategy Comparison:**

Token stream: `[A, B, C, D, E, F, G, H, I, J]`

**Fixed batch** (current): `.batch(4)`
- Chunk 1: `[A, B, C, D]` (tokens 0-3)
- Chunk 2: `[E, F, G, H]` (tokens 4-7)
- Chunk 3: `[I, J]` (tokens 8-9)
- → No overlap, each token appears once

**Sliding window**: `.window(size=4, shift=2, drop_remainder=True)`
- Chunk 1: `[A, B, C, D]` (tokens 0-3)
- Chunk 2: `[C, D, E, F]` (tokens 2-5, overlaps last 2 from chunk 1)
- Chunk 3: `[E, F, G, H]` (tokens 4-7, overlaps last 2 from chunk 2)
- → Overlap preserves context across chunks, useful for long documents

**IMPORTANT: Both strategies train on next-token prediction at EVERY position!**

**Fixed batch:**
- Chunk 1: `[A,B,C,D]` → trains: (A→B), (A,B→C), (A,B,C→D)
- Chunk 2: `[E,F,G,H]` → trains: (E→F), (E,F→G), (E,F,G→H)
- Each token appears ONCE

**Sliding window:**
- Chunk 1: `[A,B,C,D]` → trains: (A→B), (A,B→C), (A,B,C→D)
- Chunk 2: `[C,D,E,F]` → trains: (C→D), (C,D→E), (C,D,E→F)
- Tokens C and D appear TWICE (extra training for better context)

**Key Takeaway:** The chunking method only affects which tokens are grouped together, not how training works. Sliding window gives overlapping tokens extra exposure for better long-range dependencies.

## 2 Embedding and Unembedding

### 2.1 Word embedding lookup tables

**What a Embedding is**

- An embedding is just a trainable lookup table:
  - Shape: (vocab_size, d_model)
  - Input: token ID's of shape (batch, seq_len)
  - Output: vectors of shape (batch, seq_len, d_model)

Each token ID indexes one row, the learning happens because gradients update those rows based on prediction error

**Traning Proccess**: How Embeddings Are Learned

1. **Random Initialization**: Embedding table starts with random values (shape: `vocab_size × d_model`)

2. **Forward Pass**: 
    - Token IDs → Look up embeddings → Feed through transformer → Predict next token

3. **Loss Calculation**:
    - Compare prediction to actual next token
    - Compute cross-entropy loss

4. **Backpropagation**:
    - Gradients flow back through the entire model
    - Embedding table rows get gradient updates based on which tokens were used

5. **Update Rule** (simplified):
    ```
    embedding[token_id] -= learning_rate × gradient[token_id]
    ```

**Key Insight**: Tokens that appear in similar contexts will develop similar embeddings because they receive similar gradient updates. For example:
- "cat" and "dog" → often surrounded by words like "pet", "animal" → embeddings become similar
- "king" and "queen" → share contexts like "royal", "throne" → learn related representations

The model learns embeddings **jointly** with all other parameters (attention weights, feedforward layers) to minimize prediction error across the entire training corpus.

**Example Matrix:**
``` text
vocab_size = 50000  # Total unique tokens in vocabulary
d_model = 512       # Each token → 512-dimensional vector
embedding_table = tf.Variable(shape=(50000, 512))
# If token ID = 42, its embedding is embedding_table[42] (a 512-length vector)
```

In short the embedding is the models repersentation of a token, attention uses these repersentations to operate

**TensorFlow Example**
``` py
import tensorflow as tf

class TokenEmbedding(tf.keras.layers.Layer): # inherit tf Layer Class
    def __init__(self, vocab_size, d_model): # init model
        super().__init__()
        self.embedding = tf.keras.layers.Embedding( # create embedding layer
            input_dim=vocab_size,
            output_dim=d_model
        )

    # override call function with out custom embeddign layer
    def call(self, token_ids):
        return self.embedding(token_ids)
```

Example IO:
```text
input:  [12,   431,   98] # token ID's

# each token gets a embedding returned (output[i] = E[token_ids[i]]) where 'E' is teh embedding matrix
output: [
  E[12],     # ∈ R^d_model here 'R' is (real numbers) and 'd_model' is the number of dimentions in the model its a hyperparameter you chose (like 64, 128, 512, etc) for ex R^4 = [x1, x2, x3, x4] x can be any real number
  E[431],    # ∈ R^d_model
  E[98]      # ∈ R^d_model
]


```




### 2.2 Unembedding and tied embeddings

#### 2.2.1 Unembedding

Unembedding converts each transformer output vector into a score for every token in the vocabulary so the model can predict which token comes next. we need this at the output of the transformer
```
Vocabulary: ["hello", "world", "cat", "dog", ...] (ex 50000 words)
                  ↓       ↓      ↓     ↓
Logits:          [2.1,   0.5,  -1.2,  3.4, ...]  ← Higher score = more likely next token (50000 scores one for each token)
```
this is done with linear projection.

- Input: (batch, seq_len, d_model)
- Weight: (d_model, vocab_size)
- Output: (batch, seq_len, vocab_size)

**NOTE:** this layer is also called the unembedding or LM head

**NOTE:** The converstion of the token ID back to a final word is the tokenizers job do not mistake unebedding for that


**Training Process**: How Unembedding Weights Are Learned

1. **Random Initialization**: Unembedding matrix starts with random values (shape: `d_model × vocab_size`)

2. **Forward Pass**:
    - Transformer outputs → (batch, seq_len, d_model)
    - Matrix multiply with unembedding weights → (batch, seq_len, vocab_size)
    - Apply softmax → probability distribution over vocabulary

3. **Loss Calculation**:
    - Compare predicted probabilities to actual next token (one-hot encoded)
    - Compute cross-entropy loss: `loss = -log(P(correct_token))`

4. **Backpropagation**:
    - Gradients flow back from loss through softmax and unembedding layer
    - Each row of the unembedding matrix (corresponding to one output dimension) gets updated based on prediction errors

5. **Update Rule** (simplified):
    ```
    unembedding_weights -= learning_rate × gradient
    ```

**Key Insight**: The unembedding layer learns which transformer output patterns correspond to which tokens. If the model frequently outputs vectors in a certain direction when "cat" should be next, those weights get strengthened to produce higher logits for "cat".

**Example:**
``` text
d_model = 512
vocab_size = 50000
unembedding_matrix = tf.Variable(shape=(512, 50000))

# Transformer output: (batch=1, seq_len=1, d_model=512)
# After matmul: (batch=1, seq_len=1, vocab_size=50000)
# Each position gets 50000 scores (one per possible next token)

# Mathamatically

# Hidden vector h ∈ R^d_model # from tansfromer 
# Unembedding matrix W ∈ R^(d_model × vocab_size) # learned matrix
# Logits l = h · W   → l ∈ R^vocab_size # 50000 logits as a result
```

**The flow so far is as follows**: Token ID → Embedding → Transformer → Hidden Vector → Unembedding → Logits → Token ID → Word (where we pick the highest logit as next word)

**Tensorflow Example**

```py
import tensorflow as tf

class TokenUnembedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        # Linear layer without bias: projects hidden vectors to vocab logits
        self.dense = tf.keras.layers.Dense(vocab_size, use_bias=False)

    def call(self, hidden_states):
        # hidden_states: (batch, seq_len, d_model)
        # logits: (batch, seq_len, vocab_size)
        logits = self.dense(hidden_states)
        return logits

```

#### 2.2.2 Tied Embeddings

Modern LLMs tie the input embedding matrix and output projection weights 

- Why it works:
  - the same geometric space is used for reading and writing tokens 
  - reduces parameters 
  - improves sample efficiency and stability
- Mathamatically
  - input embedding: E[token_id]
  - output logits: h · Eᵀ (h is the hidden vector)
  
This enforces symmetry between encoding and decoding

**Tensorflow example**
``` py
class TiedOutputProjection(tf.keras.layers.Layer):
    def __init__(self, embedding_layer):
        super().__init__()
        self.embedding_layer = embedding_layer

    def call(self, hidden_states):
        embedding_matrix = self.embedding_layer.embedding.embeddings
        logits = tf.einsum("btd,vd->btv", hidden_states, embedding_matrix)
        return logits
```

### 2.3 Why positional representations are required (ML notes pg 112)

**Self attention is permutation-invariant.**

That means:
- “cat sat mat” and “mat sat cat” look identical without position.
- Order must be injected explicitly.

Positions noting sequence index are added or applied to embeddings before attention.

**Key idea:**
Token meaning + position meaning = input representation


### 2.4 Positional encoding types

#### 2.4.1 Sinusodal positional encoding (ML Notes pg 112)

**Concept**
- Deterministic, non-trainable
- Uses sine and cosine at different frequencies
- Allows extrapolation to longer sequences

**Formulas:**
- PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
- PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))

**Why it works:**
- Relative positions can be inferred from linear combinations
- Attention can compute distance relationships

**Tensorflow Example:**
```py
import math

def sinusoidal_position_encoding(seq_len, d_model):
    position = tf.range(seq_len, dtype=tf.float32)[:, None]
    div_term = tf.exp(
        tf.range(0, d_model, 2, dtype=tf.float32) *
        -(math.log(10000.0) / d_model)
    )

    pe = tf.zeros((seq_len, d_model))
    pe = tf.tensor_scatter_nd_update(
        pe,
        indices=tf.range(0, d_model, 2)[:, None],
        updates=tf.sin(position * div_term)
    )
    pe = tf.tensor_scatter_nd_update(
        pe,
        indices=tf.range(1, d_model, 2)[:, None],
        updates=tf.cos(position * div_term)
    )
    return pe
```

**added directly to token embeddings**

#### 2.4.2 Learned positional embeddings (ML notes pg 112)
**Concept**
- Position IDs get their own embedding table
- Fully learned
- Used in GPT-2, BERT

**Pros:**
- Flexible
- Often better on fixed context lengths

**Cons:**
- Cannot extrapolate beyond max length trained

**Tensorflow Example:**
```py
class LearnedPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(max_len, d_model)

    def call(self, seq_len):
        positions = tf.range(seq_len)
        return self.embedding(positions)
```

#### 2.4.3 Rotary Position Embeddings (RoPE)
This is where modern LLMs diverge from early transformers.

**Core idea:** RoPE rotates query and key vectors in embedding space based on position.

**Key properties:**
- Position information is applied inside attention
- Enables relative position reasoning
- Scales well to long contexts

Instead of adding position vectors, we rotate pairs of dimensions:
```text
embedding vector = [x1, x2] → rotation by angle θ( determined by tokens position)
x1' = x1*cosθ - x2*sinθ
x2' = x1*sinθ + x2*cosθ
x_rotated = [x1', x2']
```

**Why this is powerful**
- Dot products encode relative distance naturally when dot product of K,Q are rotated
- No learned position embeddings
- Better extrapolation

**TensorFlow implementation**
```py
def rotary_embedding(x, seq_len):
    d_model = x.shape[-1]
    half = d_model // 2

    freqs = tf.exp(
        -tf.range(0, half, dtype=tf.float32) / half * tf.math.log(10000.0)
    )
    positions = tf.range(seq_len, dtype=tf.float32)
    angles = positions[:, None] * freqs[None, :]

    sin = tf.sin(angles)
    cos = tf.cos(angles)

    x1 = x[..., :half]
    x2 = x[..., half:]

    rotated = tf.concat(
        [x1 * cos - x2 * sin,
         x1 * sin + x2 * cos],
        axis=-1
    )
    return rotated
```
**Applied to queries and keys only, never values.**

#### 2.4.4 ALiBi
**Concept**
- No position embeddings at all
- Adds a linear bias to attention scores
- Penalizes distant tokens

**Attention score becomes:**
``` text
QKᵀ / sqrt(d) + bias(distance)
```

**Why it matters:**
- Extremely simple
- Strong extrapolation to long sequences
- Used in MPT and others

**NOTE:** ALiBi changes attention, not embeddings.

**You do not add position vectors at input.**




### 2.5 How positional encoding interacts with attention
**Important clarity:**
- Additive encodings modify token representations before attention.
- RoPE modifies query and key geometry.
- ALiBi modifies attention logits directly.

**All three inject order, but at different stages.**

**This choice affects:**
- Long context scaling
- Memory behavior
- Generalization beyond training length

### 2.6 Embedding scaling by sqrt(d_model)

When embeddings are initialized, their variance is small.

If we add positional encodings directly, they can dominate early training.

**Standard fix:**
```text
x = embedding(token_ids) * sqrt(d_model)
x = x + position_encoding
```

**This ensures:**
- Token identity dominates initially
- Position is a refinement, not the signal

**TF snippet**
```py
x = token_embedding(token_ids)
x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
x += position_encoding
```

**This is not cosmetic. It stabilizes training.**


### 2.7 Complete Example: Combining All Embedding Steps

This example demonstrates the entire embedding pipeline from token IDs to final logits, a code example for 2.1-2.6.

In [12]:
import tensorflow as tf
import keras_nlp

# Load pretrained tokenizer and model
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
backbone = keras_nlp.models.GPT2Backbone.from_preset("gpt2_base_en")

# Example text: Notice "cat" appears at different positions
text = "The cat sat on the mat. I love my cat very much."
print(f"Text: {text}\n")

# Step 1: Tokenize (from section 1)
token_ids = tokenizer(text)
print(f"1. Token IDs: {token_ids.numpy()}\n")

# Step 2: Token Embedding (2.1) - Convert IDs to vectors
token_embedding = backbone.token_embedding
token_embeds = token_embedding(token_ids)
print(f"2. Token Embeddings shape: {token_embeds.shape}")
print(f"   Each token → 768-dimensional vector\n")

# Step 3: Position Embedding (2.4) - Add position information
position_embedding = backbone.position_embedding
pos_embeds = position_embedding(token_embeds)
print(f"3. Position Embeddings shape: {pos_embeds.shape}\n")

# Step 4: Combine (2.5) - Token meaning + Position
final_embeds = token_embeds + pos_embeds
print(f"4. Final Embeddings (token + position): {final_embeds.shape}\n")

# ============================================================
# KEY DEMONSTRATION: Same word, different position
# ============================================================
cat_token = tokenizer("cat")[0]
cat_positions = [i for i, t in enumerate(token_ids.numpy()) if t == cat_token.numpy()]

if len(cat_positions) >= 2:
    pos1, pos2 = cat_positions[0], cat_positions[1]
    
    print(f"Word 'cat' appears at positions: {cat_positions}")
    print(f"\nPosition {pos1}:")
    print(f"  Token embed (first 3): {token_embeds[pos1, :3].numpy()}")
    print(f"  Pos embed (first 3):   {pos_embeds[pos1, :3].numpy()}")
    print(f"  Final embed (first 3): {final_embeds[pos1, :3].numpy()}")
    
    print(f"\nPosition {pos2}:")
    print(f"  Token embed (first 3): {token_embeds[pos2, :3].numpy()}")
    print(f"  Pos embed (first 3):   {pos_embeds[pos2, :3].numpy()}")
    print(f"  Final embed (first 3): {final_embeds[pos2, :3].numpy()}")
    
    print("\n✓ Token embeddings: IDENTICAL (same word)")
    print("✓ Position embeddings: DIFFERENT (different locations)")
    print("✓ Final embeddings: DIFFERENT (context-aware)")

# ============================================================
# Step 5: Unembedding (2.2) - Project back to vocabulary
# ============================================================
print(f"\n5. Unembedding (tied weights):")
# NOTE: We're using GPT-2's PRETRAINED embedding weights
# These were learned on billions of tokens, so predictions are meaningful
# In practice, you'd train these from scratch on your data
embedding_matrix = token_embedding.embeddings  # Reuse same pretrained weights
logits = tf.matmul(final_embeds, embedding_matrix, transpose_b=True)
print(f"   Logits shape: {logits.shape}")
print(f"   → {logits.shape[0]} positions × {logits.shape[1]} vocab scores")
print(f"   (Using pretrained GPT-2 embeddings - trained on web text)")

# Show top predictions for first few positions
print(f"\nTop predicted tokens (first 5 positions):")
for i in range(min(5, len(token_ids))):
    predicted_id = tf.argmax(logits[i]).numpy()
    predicted_word = tokenizer.detokenize([predicted_id])
    print(f"  Position {i}: '{predicted_word}'")

Text: The cat sat on the mat. I love my cat very much.

1. Token IDs: [ 464 3797 3332  319  262 2603   13  314 1842  616 3797  845  881   13]

2. Token Embeddings shape: (14, 768)
   Each token → 768-dimensional vector

3. Position Embeddings shape: (14, 768)

4. Final Embeddings (token + position): (14, 768)


5. Unembedding (tied weights):
   Logits shape: (14, 50257)
   → 14 positions × 50257 vocab scores
   (Using pretrained GPT-2 embeddings - trained on web text)

Top predicted tokens (first 5 positions):
  Position 0: 'theless'
  Position 1: ' cat'
  Position 2: ' sat'
  Position 3: ' on'
  Position 4: ' the'


**SUMMARY: What we covered from sections 2.1-2.6**

**1. TOKEN EMBEDDING (2.1):**
- Input: Token IDs [464, 3797, ...]
- Output: Dense vectors [(768 dims), (768 dims), ...]
- → Each token gets a learned representation

**2. POSITION EMBEDDING (2.4):**
- Learned embeddings that encode position
- Position 0 ≠ Position 10 (different vectors)
- → Tells model WHERE each token is

**3. COMBINING (2.5):**
- final = token_embed + pos_embed
- → Same word at different positions has different final representations

**4. UNEMBEDDING (2.2):**
- Projects embeddings → vocabulary logits
- Uses TIED WEIGHTS (same matrix as input embeddings)
- → Predicts next token distribution

**IMPORTANT:** This example uses GPT-2's PRETRAINED embeddings!
- These weights were learned on billions of tokens of web text
- That's why predictions are meaningful (not random)
- In your own model, you'd train these from scratch on your data
- Training process: same backpropagation as described in 2.1

**Key Insight:** Without position embeddings, "cat sat mat" and "mat sat cat" would look identical to the model. Position info is CRITICAL.

**Note:** We haven't covered transformers yet - that processes these embeddings! The transformer would sit between step 4 (final embeddings) and step 5 (unembedding).

## 3 Attention (ML notes pg 54-61)

#### 3.1 Query, Key, Value fundamentals
Every token embedding is projected into three different spaces:
- Query (Q): what this token is looking for
- Key (K): what this token offers
- Value (V): the information this token contributes if selected

All three come from the same input embedding x, but with different learned linear projections. The input token starts as the same vector x and is transformed using a learned weight matrix

**Mathematically:**
- Q = x Wq
- K = x Wk
- V = x Wv
  
Why this works:
- Attention becomes content-addressable memory.
- Tokens don’t attend by position or index, but by similarity in meaning.

Conceptual analogy:
- Query = question
- Key = label on a memory slot
- Value = content inside the slot


### 3.2. Dot product attention

The core operation:
```math
attention(Q, K, V) = softmax(Q Kᵀ / √dk) V
```

Step by step:
1. Compute similarity between every query and every key.
2. Normalize scores with softmax.
3. Use scores to form weighted sums of values.

This produces:
- For each token, a contextualized representation.
- Tokens can pull information from any previous token.

### 3.3 Why divide by √dk

Without scaling:
- Dot products grow with dimension.
- Softmax saturates (becomes near one-hot).
- Gradients vanish.

Scaling by **sqrt(dk)** keeps variance stable.

This is not optional. Training becomes unstable without it.


### 3.4 Causal masking and autoregressive behavior

LLMs generate left to right. A token must not see future tokens during training.

Causal mask:
- Upper triangular matrix filled with -inf
- Added to attention logits before softmax

Effect:
- Positions j > i get zero probability
- Guarantees autoregressive behavior

Conceptually:
- Training simulates generation
- Prediction at position i only depends on < i

**Similarity Scores Q Kᵀ (no mask)**
| Query \ Key | I    | love | ML   |
|------------|------|------|------|
| I          | 0.60 | 0.30 | 0.10 |
| love       | 0.20 | 0.50 | 0.30 |
| ML         | 0.10 | 0.67 | 0.23 |

**Casual mask**
| Query \ Key | I    | love | ML   |
|------------|------|------|------|
| I          | 1.00 | 0.00 | 0.00 |
| love       | 0.29 | 0.71 | 0.00 |
| ML         | 0.10 | 0.67 | 0.23 |

### 3.5 Softmax details and numerical stability

Softmax is:
$$
\text{softmax}(x_i) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}
$$

Numerical issue:
- Large logits overflow

Standard trick:
$$
\text{softmax}(x_i) = \frac{\exp(x_i - \max_j x_j)}{\sum_j \exp(x_j - \max_j x_j)}
$$
This preserves probabilities but stabilizes computation.

TensorFlow handles this internally, but you must remember why it works.

### 3.6 Multi-head attention
Single head attention has one similarity space basicaly it can only attend to one type of question. see `ML notes pg 61` for more

Multi-head attention:
- Splits channels into h heads
- Each head attends differently
- Results are concatenated and projected

Shape intuition:
```
(batch, seq, d_model)
→ (batch, heads, seq, d_head)
```
Where:
```
d_head = d_model / heads
```

Why this matters:
- Different heads learn syntax, semantics, coreference, long-range dependencies
- Heads act as independent subspaces

**Example**
Tokens: T1, T2, T3

Head 1 output:
| T1 | T2 | T3 |
|----|----|----|
| 0.1 | 0.3 | 0.6 |
| 0.2 | 0.5 | 0.3 |

Head 2 output:
| T1 | T2 | T3 |
|----|----|----|
| 0.4 | 0.4 | 0.2 |
| 0.3 | 0.3 | 0.4 |

Concatenated (along features):
| T1         | T2         | T3         |
|------------|------------|------------|
| 0.1, 0.4   | 0.3, 0.4   | 0.6, 0.2   |
| 0.2, 0.3   | 0.5, 0.3   | 0.3, 0.4   |

**Multi-Head Attention Combination**

Each head computes its own attention output independently:
$$
\text{head}_i = \text{Attention}(Q_i, K_i, V_i)
$$

Outputs of all heads are concatenated along the feature dimension:
$$
\text{Concat}(\text{head}_1, \ldots, \text{head}_h) \in \mathbb{R}^{d_{\text{model}}}
$$

A final learned linear projection mixes them:
$$
\text{MHA\_output} = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) \cdot W_O
$$

Note: attention scores are not combined across heads; each head attends separately.

### 3.7 Attention complexity

Time and memory:
```O(seq_len²)```

This is the main scaling bottleneck in LLMs.

Implications:
- Long context is expensive
- Motivates FlashAttention, sparse attention, sliding windows


### 3.8 Memory layout and tensor shapes (critical)

Most bugs in attention come from shape mistakes.

Typical shapes:
- Q, K, V: (batch, heads, seq_len, d_head) # d_head = d_model / heads
- Attention scores: (batch, heads, seq_len, seq_len)
- Output: (batch, seq_len, d_model)

Reshaping and transposing correctly is essential.


### 3.9 Flash Attention (conceptual only)

Flash Attention:
- Computes attention without materializing full seq_len × seq_len matrix
- Uses tiling and fused kernels
- Reduces memory bandwidth bottleneck

Important takeaway:
- Same math, different execution
- You do not change the model, only the kernel


### 3.10 Self attention vs cross attention

- Self attention: Q, K, V come from same sequence
- Cross attention: Q from decoder, K/V from encoder

GPT style LLMs use only self attention.

Encoder–decoder models use both.


### 3.11 Key-value caching for generation

During autoregressive generation:
- Past K and V do not change
- Only compute Q for new token
- Append new K, V to cache

Effect:
- Reduces per-token cost from O(n²) to O(n)


This is essential for fast inference.


### 3.12 Tensorflow Example (no raw kernal high level usage)

#### 3.12.1 Single-head attention (conceptual TF)

In [1]:
import tensorflow as tf

def single_head_attention(x, mask=None):
    d_model = x.shape[-1]

    q = tf.keras.layers.Dense(d_model)(x)
    k = tf.keras.layers.Dense(d_model)(x)
    v = tf.keras.layers.Dense(d_model)(x)

    scores = tf.matmul(q, k, transpose_b=True)
    scores /= tf.math.sqrt(tf.cast(d_model, tf.float32))

    if mask is not None:
        scores += (mask * -1e9)

    weights = tf.nn.softmax(scores, axis=-1)
    output = tf.matmul(weights, v)
    return output

#### 3.12.2 Multi-head attention (high level)

In [2]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,
            dropout=dropout
        )

    def call(self, x, mask=None, training=False):
        return self.mha(
            query=x,
            value=x,
            key=x,
            attention_mask=mask,
            training=training
        )


#### 3.12.3 Causal mask

In [3]:
def causal_mask(seq_len):
    mask = tf.linalg.band_part(
        tf.ones((seq_len, seq_len)), -1, 0
    )
    return 1.0 - mask

#### 3.12.4 Multi-Head Attention in Action: Complete Demo

This example demonstrates multi-head attention processing with:
- **Embeddings before/after attention** - See how attention updates representations
- **Attention scores** - Visualize which tokens attend to which
- **Per-head analysis** - See what different heads learn

In [19]:
import tensorflow as tf
import keras_nlp
import numpy as np

# Load pretrained GPT-2
tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
backbone = keras_nlp.models.GPT2Backbone.from_preset("gpt2_base_en")

# Interesting sentence with clear relationships
text = "The cat chased the mouse"
print(f"Text: {text}\n")

# Tokenize
token_ids = tokenizer(text)
tokens = [tokenizer.detokenize([tid]).numpy().decode() if isinstance(tokenizer.detokenize([tid]), tf.Tensor) 
          else tokenizer.detokenize([tid]) for tid in token_ids.numpy()]
print(f"Tokens: {tokens}\n")

if tf.rank(token_ids) == 1:
    token_ids = token_ids[None, :]

# Get embeddings
x_tokens = backbone.token_embedding(token_ids)
x_pos = backbone.position_embedding(x_tokens)
embeddings = x_tokens + x_pos

print("="*70)
print("PRETRAINED GPT-2: Embeddings Before vs After Transformer")
print("="*70)

# Show embeddings BEFORE transformer processing
print("\nEmbeddings BEFORE attention (first 3 dims of each token):")
for i, token in enumerate(tokens):
    print(f"{token:<12} [{embeddings[0, i, 0].numpy():.3f}, {embeddings[0, i, 1].numpy():.3f}, {embeddings[0, i, 2].numpy():.3f}, ...]")

# Pass through FIRST transformer layer (uses pretrained attention weights!)
transformer_layer = backbone.transformer_layers[0]
output_embeddings = transformer_layer(embeddings)

print("\nEmbeddings AFTER pretrained attention (first 3 dims):")
for i, token in enumerate(tokens):
    print(f"{token:<12} [{output_embeddings[0, i, 0].numpy():.3f}, {output_embeddings[0, i, 1].numpy():.3f}, {output_embeddings[0, i, 2].numpy():.3f}, ...]")

# Show the magnitude of change
print("\n" + "="*70)
print("HOW MUCH EACH TOKEN CHANGED:")
print("="*70)
for i, token in enumerate(tokens):
    change = tf.norm(output_embeddings[0, i] - embeddings[0, i]).numpy()
    bar = "█" * int(change * 2)
    print(f"{token:<12} {change:.3f} {bar}")

# ============================================================
# NEXT WORD PREDICTION (after just 1 layer)
# ============================================================
print("\n" + "="*70)
print("NEXT WORD PREDICTION (using 1-layer output):")
print("="*70)

# Unembed: project back to vocabulary
embedding_matrix = backbone.token_embedding.embeddings
logits = tf.matmul(output_embeddings, embedding_matrix, transpose_b=True)

# Get top 3 predictions for the LAST position (next word)
last_logits = logits[0, -1, :]
top3_ids = tf.argsort(last_logits, direction='DESCENDING')[:3]

print("Top 3 predicted next tokens:")
for i, token_id in enumerate(top3_ids.numpy(), 1):
    token = tokenizer.detokenize([token_id])
    if isinstance(token, bytes):
        token = token.decode('utf-8')
    score = last_logits[token_id].numpy()
    print(f"  {i}. '{token}' (score: {score:.1f})")

print("\n✓ These predictions use only 1 transformer layer")
print("✓ GPT-2 has 12 layers - predictions improve with each layer!")
print("✓ Scores are logits (unnormalized) - higher = more likely")

Text: The cat chased the mouse

Tokens: ['The', ' cat', ' chased', ' the', ' mouse']

PRETRAINED GPT-2: Embeddings Before vs After Transformer

Embeddings BEFORE attention (first 3 dims of each token):
The          [-0.087, -0.218, 0.068, ...]
 cat         [0.034, -0.017, 0.069, ...]
 chased      [0.015, -0.164, 0.209, ...]
 the         [-0.040, -0.069, 0.148, ...]
 mouse       [0.120, 0.047, 0.155, ...]

Embeddings AFTER pretrained attention (first 3 dims):
The          [0.156, -0.795, 0.394, ...]
 cat         [-0.269, -0.858, 0.098, ...]
 chased      [-1.100, -1.250, -1.771, ...]
 the         [-1.145, 0.412, -0.563, ...]
 mouse       [1.029, -1.552, -0.271, ...]

HOW MUCH EACH TOKEN CHANGED:
The          130.797 █████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
 cat         5

#### 3.12.5 What Transformers Do and what do (context integration)
**The transformer's job when updating embeddings:**

Each transformer layer refines token embeddings by **integrating context from surrounding tokens** via attention:
- Before: Each token's embedding is isolated (just word + position info)
- After: Each token's embedding incorporates information from relevant context
- Example: "mouse" embedding gets updated based on "cat chased" → now encodes "something being chased"

**Why this matters for prediction:**

Updated embeddings → better next-token predictions. GPT-2 stacks 12 layers because:
- **1 layer**: Minimal context integration → weak predictions
- **12 layers**: Deep reasoning across all context → strong predictions

**Logit scores:** Higher number = more likely next token (passed through softmax to get probabilities)